In [55]:
# Topics
# This mini project is dedicated to following topics:

# Data Wrangling
# Data Visualization
# Data Preparation and Feature Engineering
# Dimensionality Reduction
# Unsupervised Learning
# Data
# We will be using old data about different financial transactions. You can download the data from here. The data contains following tables:

# twm_customer - information about customers
# twm_accounts - information about accounts
# twm_checking_accounts - information about checking accounts (subset of twm_accounts)
# twm_credit_accounts - information about checking accounts (subset of twm_accounts)
# twm_savings_accounts - information about checking accounts (subset of twm_accounts)
# twm_transactions - information about financial transactions
# twm_savings_tran - information about savings transactions (subset of twm_transactions)
# twm_checking_tran - information about savings transactions (subset of twm_transactions)
# twm_credit_tran - information about credit checking (subset of twm_transactions)
# Output
# In this miniproject, we will:

# create two separate customer segmentations (using clustering) to split them into 3-5 clusters:
# based on demographics (only on the information from twm_customer)
# based on their banking behavior. We can take following things into consideration as banking behavior:
# do they have savings account? How much do they save?
# do they have credit account? How much do they live in debt?
# are they making lot of small transactions or few huge ones?
# visualize the created clusters using radar charts and compare them agains each other
# visualize segmentations using scatter plot. We will have to use PCA to be able to plot our observations in 2D.
# (stretch) visualize in 2D how our clusters are evolving in each iteration of KMeans (for at least 20 iterations).
# we will need to create own implementation of kmeans so we can see what is happening with the clusters during the iterations.

In [56]:
import pandas as pd
import numpy as np
 

In [57]:
transactionDF = pd.read_csv('twm_transactions.csv', ';')
credTransDF = pd.read_csv('twm_credit_tran.csv', ';')
savTransDF = pd.read_csv('twm_savings_tran.csv', ';')
checkTransDF = pd.read_csv('twm_checking_tran.csv', ';')
acctsDF = pd.read_csv('twm_accounts.csv', ';')

In [58]:
transactionDF.head()

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
0,27,13625623,0.00,0.00,0.0,3753.34,21.10.1995,121656,A,IQ
1,97,13628392,0.00,0.00,0.0,254.49,5.2.1995,153053,V,IQ
2,21,13630842,-97.57,-97.57,0.0,3819.56,23.7.1995,0,P,WD
3,44,13631412,-0.15,-0.15,0.0,224.05,30.1.1995,0,,FK
4,31,13625722,0.00,0.00,0.0,240.55,25.1.1995,204521,B,IQ


In [59]:
print(len(credTransDF))
print(len(savTransDF))
print(len(checkTransDF))
print(len(transactionDF))
print(len(credTransDF)+len(savTransDF)+len(checkTransDF))

20167
11189
46204
77580
77560


In [60]:
transactionDF.dtypes

tran_id            int64
acct_nbr           int64
tran_amt         float64
principal_amt    float64
interest_amt     float64
new_balance      float64
tran_date         object
tran_time          int64
channel           object
tran_code         object
dtype: object

In [61]:
transactionDF.isna().sum()

tran_id          0
acct_nbr         0
tran_amt         0
principal_amt    0
interest_amt     0
new_balance      0
tran_date        0
tran_time        0
channel          0
tran_code        0
dtype: int64

In [62]:
transactionDF = transactionDF.sort_values('acct_nbr')

In [63]:
transactionDF

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
58300,17,13624802,1534.10,1534.10,0.00,1616.99,6.1.1995,94746,A,DP
13412,49,13624802,-358.06,-358.06,0.00,48.06,19.1.1995,0,P,WD
23044,63,13624802,-25.32,-25.32,0.00,25.31,24.1.1995,200657,A,TR
69300,82,13624802,-23.42,-23.42,0.00,23.41,14.2.1995,212713,A,TR
45824,64,13624802,-12.66,-12.66,0.00,12.65,25.1.1995,0,P,WD
...,...,...,...,...,...,...,...,...,...,...
27594,22,4561143213634920,-635.13,-635.13,0.00,-7100.00,15.8.1995,193900,E,CG
69233,13,4561143213634920,707.57,633.35,74.22,-6466.65,7.4.1995,0,M,PM
74999,8,4561143213634920,0.00,0.00,0.00,-5949.69,5.3.1995,94942,A,IQ
15801,19,4561143213634920,656.31,565.16,91.15,-5906.78,1.7.1995,105033,C,PM


In [64]:
acctsDF.head()

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,13628063,1362806,SV,Y,10.12.1995,NaN,1430.22,284.58
1,4561143213627090,1362709,CC,Y,15.3.1993,NaN,266.34,496.15
2,4561143213628360,1362836,CC,Y,18.3.1992,NaN,55.90,1000.00
3,13633112,1363311,CK,Y,6.7.1995,NaN,11017.13,968.46
4,4561143213633610,1363361,CC,Y,17.6.1994,NaN,849.37,462.28


In [65]:
custAcctsDF = acctsDF.drop(columns=['acct_nbr', 'acct_start_date', 'starting_balance', 'ending_balance', 'acct_end_date'])

In [66]:
custAcctsDF.nunique(axis=0)

cust_id           665
acct_type           3
account_active      2
dtype: int64

In [67]:
svAcctsDF = custAcctsDF.loc[custAcctsDF['acct_type'] == 'SV  ']
ccAcctsDF = custAcctsDF.loc[custAcctsDF['acct_type'] == 'CC  ']
ckAcctsDF = custAcctsDF.loc[custAcctsDF['acct_type'] == 'CK  ']

In [68]:
svAcctsDF = svAcctsDF.loc[svAcctsDF['account_active'] == 'Y '].drop(columns=['account_active'])
ccAcctsDF = ccAcctsDF.loc[ccAcctsDF['account_active'] == 'Y '].drop(columns=['account_active'])
ckAcctsDF = ckAcctsDF.loc[ckAcctsDF['account_active'] == 'Y '].drop(columns=['account_active'])

In [69]:
svAcctsDF = svAcctsDF.rename({'acct_type':'sv_acct'}, axis=1)
ccAcctsDF = ccAcctsDF.rename({'acct_type':'cc_acct'}, axis=1)
ckAcctsDF = ckAcctsDF.rename({'acct_type':'ck_acct'}, axis=1)

In [70]:
svAcctsDF['sv_acct'] = 1
ccAcctsDF['cc_acct'] = 1
ckAcctsDF['ck_acct'] = 1

In [71]:
acctDF = pd.merge(svAcctsDF,ccAcctsDF, how='outer', on='cust_id')
acctDF = pd.merge(acctDF, ckAcctsDF, how='outer', on='cust_id')
acctDF = acctDF.fillna(0)

In [72]:
acctDF

,cust_id,sv_acct,cc_acct,ck_acct
0,1362806,1.0,1.0,1.0
1,1363377,1.0,1.0,1.0
2,1362940,1.0,1.0,0.0
3,1362823,1.0,0.0,1.0
4,1363078,1.0,1.0,1.0
...,...,...,...,...
637,1363235,0.0,0.0,1.0
638,1363415,0.0,0.0,1.0
639,1363098,0.0,0.0,1.0
640,1363434,0.0,0.0,1.0


In [18]:
savTransDF = savTransDF[['cust_id', 'tran_amt', 'tran_date']]
savTransDF['tran_date'] = pd.to_datetime(savTransDF['tran_date'])
savTransDF.sort_values(by='cust_id')
savTransDF = savTransDF.groupby(['cust_id', savTransDF['tran_date'].dt.month]).sum()
savTransDF = savTransDF.reset_index(level=[0,1])
savTransDF

,cust_id,tran_date,tran_amt
0,1362480,1,1.22
1,1362480,2,1.22
2,1362480,3,1.22
3,1362480,4,669.09
4,1362480,5,-191.90
...,...,...,...
4861,1363495,8,3.37
4862,1363495,9,296.24
4863,1363495,10,3.37
4864,1363495,11,3.38


In [73]:
monthSavDF = savTransDF.groupby(by='cust_id')['tran_amt'].mean()

In [76]:
monthSavDF

cust_id
1362480    18.550769
1362484    58.474545
1362486    -6.729600
1362487   -33.623158
1362489   -17.572273
             ...    
1363490     2.853333
1363491   -10.423889
1363492     6.563200
1363493    11.706842
1363495    12.827778
Name: tran_amt, Length: 420, dtype: float64

In [77]:
acctDF = pd.merge(acctDF, monthSavDF, how='outer', on='cust_id')

In [41]:
checkTransDF = checkTransDF[['cust_id', 'tran_amt', 'tran_date']]
checkTransDF['tran_date'] = pd.to_datetime(checkTransDF['tran_date'])
checkTransDF.sort_values(by='cust_id')
checkTransDF = checkTransDF.groupby(['cust_id', checkTransDF['tran_date'].dt.month]).sum()
checkTransDF = checkTransDF.reset_index(level=[0,1])
checkTransDF

,cust_id,tran_date,tran_amt
0,1362480,1,-109.96
1,1362480,2,-321.65
2,1362480,3,0.00
3,1362480,4,647.58
4,1362480,5,-498.51
...,...,...,...
5689,1363495,5,92.86
5690,1363495,6,-6.80
5691,1363495,7,-6.81
5692,1363495,8,-6.81


In [83]:
monthCheckDF = checkTransDF.groupby(by='cust_id')['tran_amt'].mean()
acctDF = pd.merge(acctDF, monthCheckDF, how='outer', on='cust_id')
acctDF

,cust_id,sv_acct,cc_acct,ck_acct,tran_amt_x,tran_amt_y
0,1362806,1.0,1.0,1.0,-28.641000,23.664343
1,1363377,1.0,1.0,1.0,46.878400,-26.107677
2,1362940,1.0,1.0,0.0,23.607308,NaN
3,1362823,1.0,0.0,1.0,3.050952,-9.784524
4,1363078,1.0,1.0,1.0,18.053077,-1.348788
...,...,...,...,...,...,...
655,1363191,NaN,NaN,NaN,NaN,0.665342
656,1363242,NaN,NaN,NaN,NaN,6.230808
657,1363250,NaN,NaN,NaN,NaN,-4.042323
658,1363298,NaN,NaN,NaN,NaN,-73.638283


In [42]:
credTransDF = credTransDF[['cust_id', 'tran_amt', 'tran_date']]
credTransDF['tran_date'] = pd.to_datetime(credTransDF['tran_date'])
credTransDF.sort_values(by='cust_id')
credTransDF = credTransDF.groupby(['cust_id', credTransDF['tran_date'].dt.month]).sum()
credTransDF = credTransDF.reset_index(level=[0,1])
credTransDF

,cust_id,tran_date,tran_amt
0,1362480,1,-627.82
1,1362480,2,790.56
2,1362480,3,394.10
3,1362480,4,-139.96
4,1362480,5,253.21
...,...,...,...
4752,1363492,8,-635.13
4753,1363492,9,-944.12
4754,1363492,10,-103.43
4755,1363492,11,-1193.14


In [84]:
monthCredDF = credTransDF.groupby(by='cust_id')['tran_amt'].mean()
acctDF = pd.merge(acctDF, monthCredDF, how='outer', on='cust_id')
acctDF

,cust_id,sv_acct,cc_acct,ck_acct,tran_amt_x,tran_amt_y,tran_amt
0,1362806,1.0,1.0,1.0,-28.641000,23.664343,-18.449710
1,1363377,1.0,1.0,1.0,46.878400,-26.107677,-7.445313
2,1362940,1.0,1.0,0.0,23.607308,NaN,1.499091
3,1362823,1.0,0.0,1.0,3.050952,-9.784524,146.271333
4,1363078,1.0,1.0,1.0,18.053077,-1.348788,26.281400
...,...,...,...,...,...,...,...
659,1363428,NaN,NaN,NaN,NaN,-8.781616,NaN
660,1362845,NaN,NaN,NaN,NaN,NaN,103.784595
661,1363068,NaN,NaN,NaN,NaN,NaN,51.881739
662,1363211,NaN,NaN,NaN,NaN,NaN,53.035294
